# AAI614: Data Science & its Applications

*Notebook 3.2: Practice with Data Cleaning*

<a href="https://colab.research.google.com/drive/1_hY_zv0OkRAuSrphzt72nPJAuSApzWnr?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

Exercise I. Load the following datafile from GitHub

In [ ]:
grads = pd.read_csv("https://raw.githubusercontent.com/harmanani/AAI614/main/Week%203/grads.csv")

In [ ]:
grads

,Student Name,Avg Hours Studies per Week,GPA,University,Sense of Humour (0-5),Salary
0,George,20,NaN,NYU,3.0,$40k
1,Jerry,35,3.5,Columbia,5.0,$80k
2,Elaine,55,4.0,Columbia,4.2,$60k
3,Cosmo,5,2.0,City College,2.0,$25k
4,Newman,25,2.8,City College,0.0,$50k
5,Frank,35,3.0,Festivus Uni,NaN,$40k
6,Estelle,100,3.2,Festivus Uni,1.7,$0k
7,Leo,15,2.4,Festivus Uni,0.0,$35k
8,Rachel,50,4.0,Columbia,NaN,$75k


Question 1: Identify all the outliers in the above data.  Justify your answers using objective measures.

In [ ]:
# Step 1: Inspect the dataset
print(grads.info())
print(grads.describe())

# Step 2: Select only numeric columns for outlier analysis
numeric_cols = grads.select_dtypes(include=['float64', 'int64'])

# Step 3: Detect outliers using IQR
outliers_iqr = pd.DataFrame()

for col in numeric_cols.columns:
    Q1 = grads[col].quantile(0.25)
    Q3 = grads[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = grads[(grads[col] < lower_bound) | (grads[col] > upper_bound)]
    outliers_iqr = pd.concat([outliers_iqr, outliers])

# Drop duplicates if the same row has multiple outliers
outliers_iqr = outliers_iqr.drop_duplicates()

# Display the outliers
print("Outliers detected using IQR method:")
print(outliers_iqr)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Student Name                9 non-null      object 
 1   Avg Hours Studies per Week  9 non-null      int64  
 2   GPA                         8 non-null      float64
 3   University                  9 non-null      object 
 4   Sense of Humour (0-5)       7 non-null      float64
 5   Salary                      9 non-null      object 
dtypes: float64(2), int64(1), object(3)
memory usage: 564.0+ bytes
None
       Avg Hours Studies per Week       GPA  Sense of Humour (0-5)
count                    9.000000  8.000000               7.000000
mean                    37.777778  3.112500               2.271429
std                     28.296545  0.716016               1.932800
min                      5.000000  2.000000               0.000000
25%                     20.000000  2.

Question 2: There are various data that are missing.  Fill-in the missing data or delete the rows/columns that you think you should delete.  Justify your answer

In [ ]:
# Step 1: Check for missing values
missing_counts = grads.isnull().sum()
missing_percent = (missing_counts / len(grads)) * 100
print("Missing values per column (%):")
print(missing_percent)

# Step 2: Handle missing values
# Drop columns with more than 40% missing
columns_to_drop = missing_percent[missing_percent > 40].index
grads_cleaned = grads.drop(columns=columns_to_drop)

# For numeric columns: fill with median
for col in grads_cleaned.select_dtypes(include=['float64', 'int64']).columns:
    grads_cleaned[col] = grads_cleaned[col].fillna(grads_cleaned[col].median())

# For categorical columns: fill with mode
for col in grads_cleaned.select_dtypes(include=['object']).columns:
    grads_cleaned[col] = grads_cleaned[col].fillna(grads_cleaned[col].mode()[0])

# Step 4: Confirm no missing values
print("Missing values after cleaning:")
print(grads_cleaned.isnull().sum())


Missing values per column (%):
Student Name                   0.000000
Avg Hours Studies per Week     0.000000
GPA                           11.111111
University                     0.000000
Sense of Humour (0-5)         22.222222
Salary                         0.000000
dtype: float64
Missing values after cleaning:
Student Name                  0
Avg Hours Studies per Week    0
GPA                           0
University                    0
Sense of Humour (0-5)         0
Salary                        0
dtype: int64


Columns with >40% missing are dropped because they provide little useful information and may bias results.

Numerical columns are filled with the median, which is more robust to outliers than the mean.

Categorical columns are filled with the mode, assuming the most frequent category is a reasonable guess.

This strategy keeps the dataset structure and statistical properties stable without excessive row deletion.

Question 3: Reload the data and fill-in the data using mean method as well as the frequent method.

In [ ]:
# Step 1: Reload the dataset
grads = pd.read_csv("https://raw.githubusercontent.com/harmanani/AAI614/main/Week%203/grads.csv")

# Step 2: Separate numeric and categorical columns
numeric_cols = grads.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = grads.select_dtypes(include=['object']).columns

# Step 3: Fill numeric columns with mean
for col in numeric_cols:
    grads[col] = grads[col].fillna(grads[col].mean())

# Step 4: Fill categorical columns with mode (most frequent)
for col in categorical_cols:
    grads[col] = grads[col].fillna(grads[col].mode()[0])

# Step 5: Check to confirm no missing values remain
print("Missing values after mean/mode imputation:")
print(grads.isnull().sum())


Missing values after mean/mode imputation:
Student Name                  0
Avg Hours Studies per Week    0
GPA                           0
University                    0
Sense of Humour (0-5)         0
Salary                        0
dtype: int64


Exercise II. Run the cell below to create a new dataframe called `df_miss`.  Its first column will contain some missing values.

In [ ]:
import pandas as pd
import numpy as np
import random

nrows = 10
ncols = 5

# set a seed for random number generation
np.random.seed(314)
# create an array filled with random data
data = np.array(np.random.rand(nrows, ncols))
# put the data to a pandas dataframe
df_miss = pd.DataFrame(data)
# rename the columns
df_miss.columns = ['col_'+str(ii) for ii in range(ncols)]

# randomly set some values to missing
ix0 = np.random.randint(nrows, size=3)
ix1 = np.random.randint(nrows, size=3)

df_miss['col_0'][ix0] = np.nan
df_miss['col_1'][ix1] = np.nan

print(df_miss)

      col_0     col_1     col_2     col_3     col_4
0       NaN       NaN  0.265048  0.783205  0.918001
1  0.827355       NaN  0.260480  0.911763  0.260757
2  0.766376  0.261531  0.122291  0.386006  0.840081
3       NaN       NaN  0.633110  0.584766  0.581232
4  0.677205  0.687155  0.438927  0.320927  0.570552
5       NaN  0.861074  0.834805  0.105766  0.060408
6  0.596882  0.792395  0.226356  0.535201  0.136066
7  0.372244  0.151977  0.429822  0.792706  0.406957
8  0.177850  0.909252  0.545331  0.100497  0.718721
9  0.978429  0.309776  0.260126  0.662900  0.139720


<ipython-input-7-310065607>:21: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_miss['col_0'][ix0] = np.nan
<ipython-input-7-310065607>:22: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values t

Impute the missing values (NaN) in `col_0` (but not `col_1`) with the median.  Store the values in the dataframe by using the parameter `inplace`.  Print the dataframe.

In [ ]:
# Impute missing values in col_0 with the median, inplace
df_miss['col_0'].fillna(df_miss['col_0'].median(), inplace=True)

# Display the result
print(df_miss)


      col_0     col_1     col_2     col_3     col_4
0  0.677205       NaN  0.265048  0.783205  0.918001
1  0.827355       NaN  0.260480  0.911763  0.260757
2  0.766376  0.261531  0.122291  0.386006  0.840081
3  0.677205       NaN  0.633110  0.584766  0.581232
4  0.677205  0.687155  0.438927  0.320927  0.570552
5  0.677205  0.861074  0.834805  0.105766  0.060408
6  0.596882  0.792395  0.226356  0.535201  0.136066
7  0.372244  0.151977  0.429822  0.792706  0.406957
8  0.177850  0.909252  0.545331  0.100497  0.718721
9  0.978429  0.309776  0.260126  0.662900  0.139720


<ipython-input-10-2248949445>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_miss['col_0'].fillna(df_miss['col_0'].median(), inplace=True)


Impute the missing values in `col_1` with value 0.  Store the values in the dataframe by using the parameter `inplace`.  Print the dataframe.

In [ ]:
# Impute missing values in col_1 with 0, inplace
df_miss['col_1'].fillna(0, inplace=True)

# Display the updated dataframe
print(df_miss)


      col_0     col_1     col_2     col_3     col_4
0  0.677205  0.000000  0.265048  0.783205  0.918001
1  0.827355  0.000000  0.260480  0.911763  0.260757
2  0.766376  0.261531  0.122291  0.386006  0.840081
3  0.677205  0.000000  0.633110  0.584766  0.581232
4  0.677205  0.687155  0.438927  0.320927  0.570552
5  0.677205  0.861074  0.834805  0.105766  0.060408
6  0.596882  0.792395  0.226356  0.535201  0.136066
7  0.372244  0.151977  0.429822  0.792706  0.406957
8  0.177850  0.909252  0.545331  0.100497  0.718721
9  0.978429  0.309776  0.260126  0.662900  0.139720


<ipython-input-11-1542777383>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_miss['col_1'].fillna(0, inplace=True)
